In [2]:
import tensorflow as tf
import tensorflow_text as tf_text
import re
import numpy as np
import math

In [3]:
with open("bible.txt", "r") as f:
    text = f.read()
    text = text.lower()
    text = re.sub(r'[^a-z ]', ' ', text)
    text = text[:10000] #smaller text for testing purposes
    print(text[:1000])

splitter = tf_text.WhitespaceTokenizer()
tokens = splitter.split(text)
tokens = list(tokens.numpy())
vocabulary = {x: i for i, x in enumerate(np.unique(list(text)))}
char_tokens = [vocabulary[char] for char in text]

the first book of moses   called genesis       in the beginning god created the heaven and the earth       and the earth was without form  and void  and darkness was upon the face of the deep  and the spirit of god moved upon the face of the waters       and god said  let there be light  and there was light       and god saw the light  that it was good  and god divided the light from the darkness       and god called the light day  and the darkness he called night  and the evening and the morning were the first day       and god said  let there be a firmament in the midst of the waters  and let it divide the waters from the waters       and god made the firmament  and divided the waters which were under the firmament from the waters which were above the firmament  and it was so       and god called the firmament heaven  and the evening and the morning were the second day       and god said  let the waters under the heaven be gathered together unto one place  and let the dry land appear

In [4]:
sequence_length = 2
sequences = []
for i in range(0, len(char_tokens) - sequence_length):
    for j in range(i-sequence_length, i+sequence_length+1):
        if j == i or j < 0 or j >= len(char_tokens):
            continue
        sequences.append([char_tokens[i], char_tokens[j]])

print(sequences[:10])

[[18, 8], [18, 5], [8, 18], [8, 5], [8, 0], [5, 18], [5, 8], [5, 0], [5, 6], [0, 8]]


In [5]:
def generateDataset(sequences, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(sequences)
    dataset = dataset.shuffle(len(sequences))
    dataset = dataset.batch(batch_size)
    return dataset

In [6]:
for target in generateDataset(sequences, 10).take(1):
    print(target)

print(generateDataset(sequences, 10))

tf.Tensor(
[[16 19]
 [ 0  0]
 [ 0  0]
 [13  4]
 [ 0  0]
 [ 4  3]
 [16  8]
 [ 5 17]
 [21  0]
 [ 2  3]], shape=(10, 2), dtype=int32)
<BatchDataset element_spec=TensorSpec(shape=(None, 2), dtype=tf.int32, name=None)>


In [32]:
class SkipGram(tf.keras.Model) :
  def __init__(self, vocabulary_size, embedding_size) :
    super(SkipGram, self).__init__()
    self.vocabulary_size = vocabulary_size
    self.embedding_size = embedding_size
    self.loss = tf.nn.nce_loss
    self.build("")

  def build(self, empty) :
      self.embeddings = self.add_weight(shape=(self.vocabulary_size, self.embedding_size), initializer='random_normal', trainable=True)
      self.score_matrix = self.add_weight(shape=(self.vocabulary_size, self.embedding_size), initializer='random_normal', trainable=True)
      self.nce_biases = tf.Variable(tf.zeros([self.vocabulary_size])) 

  def call(self, inputs) :
    context = inputs[:, 0]
    target = inputs[:, 1]
    target = tf.expand_dims(target, axis=-1) #needed for nce_loss
    inputs = tf.cast(inputs, tf.int64) #needed for sampler
    target = tf.cast(target, tf.int64) #needed for sampler

    embeddings = tf.nn.embedding_lookup(self.embeddings, context)
  
    #try to solve samplers range is too small error
    samples = tf.random.log_uniform_candidate_sampler(true_classes=target, num_true=1, num_sampled=64, unique=True, range_max=self.vocabulary_size)
    
    loss = self.loss(labels=target, inputs=embeddings, weights=self.score_matrix, biases=self.nce_biases, num_sampled=64, num_classes=self.vocabulary_size, sampled_values=samples)
    return tf.reduce_mean(loss)
  


In [14]:
def train(model, dataset, optimizer, epochs):
    for epoch in range(epochs):
        for batch in dataset:
            with tf.GradientTape() as tape:
                loss = model(batch)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        print("Epoch: {}, Loss: {}".format(epoch, loss.numpy()))

In [33]:
model = SkipGram(len(vocabulary), 10)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dataset = generateDataset(sequences, 100)
train(model, dataset, optimizer, 10)



InvalidArgumentError: Exception encountered when calling layer "skip_gram_11" "                 f"(type SkipGram).

{{function_node __wrapped__LogUniformCandidateSampler_device_/job:localhost/replica:0/task:0/device:CPU:0}} Sampler's range is too small. [Op:LogUniformCandidateSampler]

Call arguments received by layer "skip_gram_11" "                 f"(type SkipGram):
  • inputs=tf.Tensor(shape=(100, 2), dtype=int32)

# no time to fix countless errors